In [19]:
import os
import openai
import speech_recognition as sr
import time
import json

Read API Keys

In [20]:
with open('openai_key.txt') as f:
    openai.api_key = f.read()

with open('google_key.txt') as f:
    GOOGLE_SPEECH_KEY = f.read()

In [21]:
STARTING_PROMPT = 'What would you like to talk about today?'

In [22]:
# Initialize recognizer and microphone
r = sr.Recognizer()
m = sr.Microphone()

# Starting messages for the conversation
messages = [
    {"role": "system", "content": "You are an English tutor holding a conversation with a student."},
    {"role": "assistant", "content": "Hello, what do you want to talk about?"}
]

Define GPT Function Callbacks

In [23]:
END_CONVERSATION = False
def set_end_flag(end_conversation: bool):
    """Sets END_CONVERSATION to end_conversation"""
    global END_CONVERSATION
    END_CONVERSATION = end_conversation
    return json.dumps({
        "conversation_terminated": END_CONVERSATION
    })
user_score = 0

functions = [
    {
        "name": "set_end_flag",
        "description": "Ends the current conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "end_conversation": {
                    "type ": "boolean",
                    "description": "Whether or not to end the conversation"
                }
            },
            "required": ["end_conversation"]
        }
    }
]

Conversation Loop

In [24]:
print("Assistant: Hello, what do you want to talk about?")

# Conversation loop
while True:
    # Listen to user's speech
    with m as source:
        r.adjust_for_ambient_noise(source)
        print("You: (Speak now!)")
        audio = r.listen(source, phrase_time_limit=5)

    # Convert speech to text
    try:
        user_speech = r.recognize_google(audio)
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand that. Please try again.")
        continue
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        break
    
    print(user_speech)
    # Add user's message to the conversation
    messages.append({"role": "user", "content": user_speech})

    # Get a response from OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions
    )

    assistant_response = response.choices[0].message

    # Check if a function was called
    if assistant_response.get('function_call'):
        available_functions = {
            "set_end_flag": set_end_flag,
        }  # only one function in this example, but you can have multiple
        function_name = assistant_response["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(assistant_response["function_call"]["arguments"])

        function_response = function_to_call(
            end_conversation=function_args.get("end_conversation")
        )
        messages.append(assistant_response)
        messages.append({"role": "function", "name": function_name, "content": function_response})
        
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )  # get a new response from GPT where it can see the function response

        messages.append(second_response.choices[0].message)
    # Check for conversation end flag
    if END_CONVERSATION:
        break
    
    # If the function actually returned something, we'd send a second message here
    
    
    print(f"Assistant: {assistant_response.content}")
    messages.append(assistant_response)

Assistant: Hello, what do you want to talk about?
You: (Speak now!)
I'd like to talk about the weather
Assistant: Sure, I'd be happy to talk about the weather. What aspect of the weather would you like to discuss?
You: (Speak now!)
actually I'd like to end this conversation goodbye
